##### produce driver stanadings

In [0]:
%run "../../constants/configuration"

In [0]:
%run "../../utils/common_functions"

In [0]:
from pyspark.sql.functions import sum, when, col, count, desc, rank
from pyspark.window import Window

In [0]:
race_results_df = spark.read.format("delta").load(f"{silver_container_path}/race_results")

In [0]:
race_year_list = df_column_to_list(race_results_df, "race_year")

In [0]:
race_results_df = spark.read.format("delta").load(f"{silver_container_path}/race_results")\
                                                .filter(col("race_year").isin(race_year_list))

In [0]:
drivers_standing_df = race_results_df\
                      .groupBy("race_year", "driver_name", "driver_nationality")
                      .agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))

In [0]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = drivers_standing_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
merge_condition = [tgt.driver_name = src.driver_name, tgt.race_year = src.race_year]
merge_delta_data(final_df, "gold", "drivers_standing", gold_container_path, "merge_condition, "race_year")

In [0]:
#final_df.write.mode("overwrite").parquet(f"{gold_container_path}")